In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import scipy
from tqdm.auto import tqdm, trange
import xgboost as xgb
import joblib
from sklearn.model_selection import cross_validate
import zipfile

from src.model import tscv, ClippedOutputRegressor
from src.data import get_feature_cols, df_to_X_y, drop_non_features, add_lagged_features, add_as_features, df_to_X


%run constants.py

baseline_reg = joblib.load(os.path.join(MODELS_DIR, 'xgb-baseline.model'))

%matplotlib inline
print("Versions:")
print("  Python: %s" % sys.version)
for module in [pd, np, sns, sklearn]:
    print("  %s: %s" %(module.__name__, module.__version__))

Versions:
  Python: 3.8.2 (default, Jul 16 2020, 14:00:26) 
[GCC 9.3.0]
  pandas: 1.1.2
  numpy: 1.19.2
  seaborn: 0.11.0
  sklearn: 0.23.2


# Model Stacking

Let's start stacking stuff. We already have some CV predictions from previous experiments, so let's start this by building a model on top of that.

In [3]:
sgd_cv_preds = pd.read_parquet(os.path.join(MODEL_OUTPUTS_DIR, 'cv-sgd-features-025.parquet'))
xgb_cv_preds = pd.read_parquet(os.path.join(MODEL_OUTPUTS_DIR, 'cv-xgb-features-025.parquet'))

In [4]:
sgd_cv_preds.head()

,oof_preds,oof_idx,item_cnt_month,fold_id
0,0.322944,2570400,0.0,0
1,0.079494,2570401,0.0,0
2,0.082675,2570402,0.0,0
3,0.082971,2570403,0.0,0
4,0.079712,2570404,0.0,0


Now the function to combine the predictions. We want to make sure the split used to generate them are the same, so let's encode that into our function too.

In [5]:
def combine_cv_preds(df1, df2, suffixes=('_df1', '_df2')):
    df = df1.merge(df2, on=['oof_idx', 'fold_id', 'item_cnt_month'], suffixes=suffixes)
    if not (df.shape[0] == df1.shape[0] and df.shape[0] == df2.shape[0]):
        raise ValueError("CV preds don't align")
    return df

In [6]:
combined_cv_preds = combine_cv_preds(sgd_cv_preds, xgb_cv_preds, suffixes=('_sgd', '_xgb'))

In [7]:
combined_cv_preds.head()

,oof_preds_sgd,oof_idx,item_cnt_month,fold_id,oof_preds_xgb
0,0.322944,2570400,0.0,0,0.583746
1,0.079494,2570401,0.0,0,0.027233
2,0.082675,2570402,0.0,0,0.106143
3,0.082971,2570403,0.0,0,0.064651
4,0.079712,2570404,0.0,0,0.120500


Let's try averaging our model outputs first.

In [8]:
combined_cv_preds['oof_preds_avg'] = (combined_cv_preds['oof_preds_xgb'] + combined_cv_preds['oof_preds_sgd']) / 2.0

In [9]:
from sklearn.metrics import mean_squared_error
def cv_df_scores(cv_df, ytrue_col='item_cnt_month', ypred_col='oof_preds'):
    scores = []
    for fold_id in cv_df['fold_id'].unique()[-3:]:
        y_true = cv_df[cv_df['fold_id'] == fold_id][ytrue_col].values
        y_pred = cv_df[cv_df['fold_id'] == fold_id][ypred_col].values
        err = mean_squared_error(y_true, np.clip(y_pred, 0, 20), squared=False)
        scores.append(err)
    return scores

In [10]:
print('XGB: %.5f' % np.mean(cv_df_scores(combined_cv_preds, ypred_col='oof_preds_xgb')))
print('SGD: %.5f' % np.mean(cv_df_scores(combined_cv_preds, ypred_col='oof_preds_sgd')))
print('Avg SGD XGB: %.5f' % np.mean(cv_df_scores(combined_cv_preds, ypred_col='oof_preds_avg')))

XGB: 0.82722
SGD: 0.89283
Avg SGD XGB: 0.84431


Which is to be expected. We can try a weighted average, but instead of guessing the weights I'll just train a linear regression with l1 penalty and positive weights on this.

First let's generate a train and validation set from the CV predictions.

In [13]:
train_set = add_as_features(combined_cv_preds, ['oof_preds_xgb', 'oof_preds_sgd'])
cv_splits = tscv.split(train_set['fold_id'], n=3, window=5, test_months=train_set['fold_id'].unique())
X, y = df_to_X_y(train_set)

In [14]:
from sklearn.linear_model import Lasso

reg = ClippedOutputRegressor(Lasso(random_state=123, alpha=1e-1, positive=True, fit_intercept=True, normalize=False))

In [15]:
scores = cross_validate(reg, X, y, cv=cv_splits, scoring='neg_root_mean_squared_error', verbose=2)
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  ................................................................
[CV] ................................................. , total=   0.1s
[CV]  ................................................................
[CV] ................................................. , total=   0.2s
[CV]  ................................................................
[CV] ................................................. , total=   0.1s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.3s finished


{'fit_time': array([0.0602026 , 0.15614223, 0.08663845]),
 'score_time': array([0.01281595, 0.00385714, 0.00487566]),
 'test_score': array([-0.78450982, -0.89489563, -0.85335484])}

In [16]:
np.mean(scores['test_score'])

-0.8442534314560913

Which is an improvement on our simple average, but not on XGB alone.

We can also try our baseline XGB on this.

In [17]:
scores = cross_validate(baseline_reg, X, y, cv=cv_splits, scoring='neg_root_mean_squared_error', verbose=2)
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=   2.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s


[CV] ................................................. , total=   1.8s
[CV]  ................................................................
[CV] ................................................. , total=   1.9s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.7s finished


{'fit_time': array([1.35984349, 1.08563542, 1.08322048]),
 'score_time': array([0.71246195, 0.71115947, 0.77305722]),
 'test_score': array([-0.78313605, -0.92053073, -0.95223111])}

Another thing we can do is the passthrough: we just re-add the previous features to our new set.

In [18]:
train_set_025 = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'train-set-features-025.parquet')).iloc[train_set['oof_idx']]

In [19]:
X_train_set_025 = df_to_X(train_set_025)
del train_set_025

In [20]:
X_passthrough = np.concatenate((X, X_train_set_025), axis=1)

In [21]:
scores = cross_validate(baseline_reg, X_passthrough, y, cv=cv_splits, scoring='neg_root_mean_squared_error', verbose=2)
scores

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  ................................................................
[CV] ................................................. , total=  11.1s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.1s remaining:    0.0s


[CV] ................................................. , total=  13.2s
[CV]  ................................................................
[CV] ................................................. , total=  13.4s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   37.7s finished


{'fit_time': array([ 9.83028913, 11.49168134, 11.72689533]),
 'score_time': array([1.23957276, 1.69191098, 1.71454096]),
 'test_score': array([-0.77580652, -0.91544785, -0.88413617])}

In [22]:
np.mean(scores['test_score'])

-0.8584635151034835

Just for diagnosing this result, let's check it against the train set alone.

In [23]:
scores = cross_validate(baseline_reg, X_train_set_025, y, cv=cv_splits, scoring='neg_root_mean_squared_error', verbose=2)
scores

[CV]  ................................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................................................. , total=  12.3s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.3s remaining:    0.0s


[CV] ................................................. , total=  13.3s
[CV]  ................................................................
[CV] ................................................. , total=  13.1s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   38.8s finished


{'fit_time': array([10.62888122, 11.5683372 , 11.94395947]),
 'score_time': array([1.68916655, 1.77262259, 1.16533637]),
 'test_score': array([-0.76964522, -0.91269864, -0.88905557])}

In [24]:
np.mean(scores['test_score'])

-0.8571331444465646

Which means adding the features do more harm than good and also that our stacked regressor is better than the features alone.

The degradation of the score in comparison with our previous baseline result is probably explained by the reduced amount of data we used for training (8 months instead of 16).